In [0]:
%run ./Includes/Copy-Datasets

In [0]:
%sql
select * from customers;

In [0]:
%sql
select from_json(profile) from customers;


In [0]:
%sql
select customer_id,profile:first_name from customers;

In [0]:
%sql
select profile from customers limit 1;

In [0]:
%sql

create or replace temp view parsed_customers
as select from_json(profile, schema_of_json('{"first_name":"Dniren","last_name":"Abby","gender":"Female","address":{"street":"768 Mesta Terrace","city":"Annecy","country":"France"}}')) as profile_struct from customers;

select * from parsed_customers;

In [0]:
%sql
describe parsed_customers;

In [0]:
%sql
select profile_struct.first_name, profile_struct.last_name from parsed_customers;

In [0]:
%sql
create or replace temp view
customers_final as
select profile_struct.*
from parsed_customers;

select * from customers_final; 


In [0]:
%sql
select order_id,customer_id,books from orders;

In [0]:
%sql
select order_id,customer_id,explode(books) from orders;

In [0]:
%sql
select customer_id,
collect_set(order_id) as orders_set,
collect_set(books.book_id) as books_set
from orders
group by customer_id;

In [0]:
%sql
select customer_id,
collect_set(books.book_id) as books_set,
array_distinct(flatten(collect_set(books.book_id))) as after_flatten
from orders
group by customer_id;

In [0]:
%sql
select * from books; 


In [0]:
%sql
select * from orders;

In [0]:
%sql
create or replace view orders_enriched as
select * from
(select *, explode(books) as books_exploded from orders) o
inner join books b on o.books_exploded.book_id = b.book_id;

select * from orders_enriched

In [0]:
%sql
  select customer_id,
         explode(books)
  from orders_enriched;

In [0]:
%sql
select * from (
  select customer_id,
         book.book_id as book_id,
         book.quantity as quantity
         --, 
  --       explode(books) as book
  from orders_enriched
  lateral view explode(books) exploded_books as book
);

In [0]:
%sql
create or replace view transactions as
select * from (
  select customer_id,
         book.book_id as book_id,
         book.quantity as quantity 
  from orders_enriched
  lateral view explode(books) exploded_books as book
)
pivot (
  sum(quantity) for book_id in ('B01', 'B02', 'B03', 'B04')
);

select * from transactions;